# init

In [1]:
import numpy as np
import pandas as pd

# load

In [105]:
path_data = "../data/"
path_raw = path_data + "raw/"
path_mid = path_data + "mid/"
path_clns = path_data + "clns/"

In [3]:
cats = pd.read_csv(path_mid+"cats.csv", index_col=0)
nums = pd.read_csv(path_mid+"nums.csv", index_col=0)
bools = pd.read_csv(path_mid+"bools.csv", index_col=0)

# clns

## fillna

##### embarked

In [7]:
cats["embarked"] = cats["embarked"].fillna("S")
cats["embarked"].isna().any()

False

##### age

In [39]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [12]:
fill_mean = nums[["age", "survived"]]
fill_mean = fill_mean[~fill_mean.survived.isna()]
fill_mean["age"] = fill_mean.age.fillna(fill_mean.age.mean())

In [13]:
fill_median = nums[["age", "survived"]]
fill_median = fill_median[~fill_median.survived.isna()]
fill_median["age"] = fill_median.age.fillna(fill_median.age.median())

In [87]:
# 線形回帰で値埋め
def zscore(x):
    m = x.mean()
    s = x.std(ddof=1)
    return (x-m)/s

X = pd.get_dummies(cats, drop_first=True)
z = zscore(nums.drop(["age", "survived"], 1))
X = X.join(z)
y = nums.age

is_na = y.isna()
y = np.log1p(y)

rgs = LinearRegression()
rgs.fit(X[~is_na], y[~is_na])

pred = rgs.predict(X[is_na])
pred = np.exp(pred)-1

base = X[~is_na]
base["age"] = np.exp(y[~is_na])-1

fill = X[is_na]
fill["age"] = pred

fill_linear = pd.concat([base, fill]).join(nums[["survived"]])
fill_linear = fill_linear[["age", "survived"]]
fill_linear = fill_linear[~fill_linear.survived.isna()]

C:\Users\keisu\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\keisu\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [88]:
def check_auc(data):
    X, y = data[["age"]], data["survived"]
    clf = LogisticRegression()
    clf.fit(X, y)
    proba = clf.predict_proba(X)[:,1]
    auc = roc_auc_score(y_true=y, y_score=proba)
    print(auc)

In [89]:
check_auc(fill_mean)

0.5329419918511361


In [90]:
check_auc(fill_median)

0.5290565799974796


In [91]:
check_auc(fill_linear)

0.5316398538245055


In [94]:
nums["age"] = nums.fillna(nums.age.mean())

## union-value

##### embarked

In [99]:
cats["embarked"] = cats.embarked.replace(["Q", "S"], "QorS")

## family-size

In [101]:
family = nums.parch + nums.sibsp
parch = nums.parch.apply(lambda x: "4+" if x >= 4 else x)
sibsp = nums.sibsp.apply(lambda x: "4+" if x >= 4 else x)

# feature engineering

## onehot-encoding

In [106]:
pd.get_dummies(cats, drop_first=True).to_csv(path_clns+"onehot_cats.csv")

In [111]:
pd.get_dummies(pd.concat([parch, sibsp], axis=1), drop_first=True).to_csv(path_clns+"onehot_parch_sibsp.csv")

In [115]:
pd.get_dummies(family, drop_first=True, prefix="family-size").to_csv(path_clns+"onehot_familysize.csv")

## target-encoding

## log, zscore